In [1]:
# BUSINESS PROBLEM

#Armut, Turkey's largest online service platform, brings together service providers and 
#those looking for services such as cleaning, remodeling and moving with just a few taps 
#on a computer or smartphone. We want to create a product recommendation system with 
#Association Rule Learning using a data set that includes users who receive services and
#the services and categories that these users have received.



In [2]:
#The Dataset Story

#This dataset captures the array of services utilized by customers, along with the service categories to which they belong.


#UserId: customer number

#ServiceId: Anonymized services belonging to each category. (Example: Sofa washing service under the cleaning category) A ServiceId can be found under different categories and refers to different services under different categories. 
           #(Example: Service with CategoryId 7 ServiceId 4 is honeycomb cleaning while service with CategoryId 2 ServiceId 4 is furniture assembly)

#CategoryId: Encrypted identifiers for service categories (For instance, categories such as house cleaning, moving services, or home improvements).

#CreateDate: The date on which the service was ordered.

In [3]:
# import Required Libraries

import pandas as pd
import numpy as np
from mlxtend.frequent_patterns import apriori, association_rules

In [4]:
# Adjusting Row Column Settings

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 500)
pd.set_option('display.expand_frame_repr', False)

In [5]:
#######################################################
#Task 1: Preparing the Data
#######################################################


#Step 1: Read the file 

df = pd.read_csv("/kaggle/input/armut-data/armut_data.csv")

In [6]:
df.head()
df.shape


(162523, 4)

In [7]:
#Step2 : Combine ServiceID and CategoryID with an underscore '_' to uniquely identify each service item.

df["Service"] = [str(row[1]) + "_" + str(row[2]) for row in df.values]


In [8]:
df.head()

,UserId,ServiceId,CategoryId,CreateDate,Service
0,25446,4,5,2017-08-06 16:11:00,4_5
1,22948,48,5,2017-08-06 16:12:00,48_5
2,10618,0,8,2017-08-06 16:13:00,0_8
3,7256,9,4,2017-08-06 16:14:00,9_4
4,25446,48,5,2017-08-06 16:16:00,48_5


In [9]:
#Step 3:Step 3: The dataset consists of the date and time when the services were received, 

#there is no basket definition (invoice etc.). In order to apply Association Rule Learning, 
#a basket (invoice etc.) definition must be created. Here, the basket definition is the services 
#that each customer receives monthly. For example; 9_4, 46_4 services received 
#by a customer with id 7256 in the 8th month of 2017 represent one basket; 9_4, 38_4 services received in the 10th month of 2017 represent
#another basket. Baskets need to be identified with a unique ID. For this, first create a new date variable containing only year and month.
#Combine UserID and the date variable you just created with “_” and assign it to a new variable called ID.

df["CreateDate"] = pd.to_datetime(df["CreateDate"])

In [10]:
df["New_Date"] = df["CreateDate"].dt.strftime("%Y-%m")

In [11]:
df.head()

,UserId,ServiceId,CategoryId,CreateDate,Service,New_Date
0,25446,4,5,2017-08-06 16:11:00,4_5,2017-08
1,22948,48,5,2017-08-06 16:12:00,48_5,2017-08
2,10618,0,8,2017-08-06 16:13:00,0_8,2017-08
3,7256,9,4,2017-08-06 16:14:00,9_4,2017-08
4,25446,48,5,2017-08-06 16:16:00,48_5,2017-08


In [12]:
df["BasketID"] = [str(row[0]) + "_" + str(row[5]) for row in df.values]

df.head()

,UserId,ServiceId,CategoryId,CreateDate,Service,New_Date,BasketID
0,25446,4,5,2017-08-06 16:11:00,4_5,2017-08,25446_2017-08
1,22948,48,5,2017-08-06 16:12:00,48_5,2017-08,22948_2017-08
2,10618,0,8,2017-08-06 16:13:00,0_8,2017-08,10618_2017-08
3,7256,9,4,2017-08-06 16:14:00,9_4,2017-08,7256_2017-08
4,25446,48,5,2017-08-06 16:16:00,48_5,2017-08,25446_2017-08


In [13]:
#######################################################
#Task 2: Produce and Propose Association Rules.
#######################################################

In [14]:
invoice_df = df.groupby(['BasketID', 'Service'])['Service'].count().unstack().fillna(0).apply(lambda x: x.map(lambda x: 1 if x > 0 else 0))

invoice_df.head()

Service,0_8,10_9,11_11,12_7,13_11,14_7,15_1,16_8,17_5,18_4,19_6,1_4,20_5,21_5,22_0,23_10,24_10,25_0,26_7,27_7,28_4,29_0,2_0,30_2,31_6,32_4,33_4,34_6,35_11,36_1,37_0,38_4,39_10,3_5,40_8,41_3,42_1,43_2,44_0,45_6,46_4,47_7,48_5,49_1,4_5,5_11,6_7,7_3,8_5,9_4
BasketID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0_2017-08,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0
0_2017-09,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0
0_2018-01,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0
0_2018-04,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
10000_2017-08,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0


In [15]:
#Step 2: Create association rules.

frequent_itemsets = apriori(invoice_df, min_support=0.01, use_colnames=True)
frequent_itemsets.head()

/opt/conda/lib/python3.10/site-packages/mlxtend/frequent_patterns/fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,support,itemsets
0,0.019728,(0_8)
1,0.026523,(11_11)
2,0.029374,(12_7)
3,0.056627,(13_11)
4,0.023406,(14_7)


In [16]:
rules = association_rules(frequent_itemsets, metric="support", min_threshold=0.01)

rules.head()


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(2_0),(13_11),0.130286,0.056627,0.012819,0.098394,1.737574,0.005442,1.046325,0.488074
1,(13_11),(2_0),0.056627,0.130286,0.012819,0.226382,1.737574,0.005442,1.124216,0.449965
2,(2_0),(15_1),0.130286,0.120963,0.033951,0.260588,2.154278,0.018191,1.188833,0.616073
3,(15_1),(2_0),0.120963,0.130286,0.033951,0.280673,2.154278,0.018191,1.209066,0.609539
4,(33_4),(15_1),0.027310,0.120963,0.011233,0.411311,3.400299,0.007929,1.493211,0.725728


In [17]:
def arl_recommender(rules_df, product_id, rec_count=1):
    sorted_rules = rules_df.sort_values("lift", ascending=False)
    recommendation_list = [] 
    for i, product in sorted_rules["antecedents"].items():
        for j in list(product): 
            if j == product_id:
                recommendation_list.append(list(sorted_rules.iloc[i]["consequents"]))
    recommendation_list = list({item for item_list in recommendation_list for item in item_list})
    return recommendation_list[:rec_count] 

In [18]:
arl_recommender(rules,"15_1", 5)

['2_0', '38_4', '15_1']